## Instituto Federal de Minas Gerais - Campus Bambuí 
### *Engenharia de Computação*

***Alunos: Gabriel Henrique Silva Duque e*** 
***Rafael Gonçalves Oliveira***

In [ ]:
import math
import logging

# --- Configurações da Árvore ---
ORDER = 3  # Ordem da Árvore B+
MAX_KEYS = ORDER - 1
MIN_KEYS = math.ceil(ORDER / 2) - 1

# --- Nó/Página da Árvore ---
class BPlusNode():
    """Representa um nó interno ou uma folha na B+ Tree."""
    def __init__(self, is_leaf=False):
        self.keys = []
        self.children = []  # Registros (se folha) ou Nós filhos (se interno)
        self.is_leaf = is_leaf
        self.next_leaf = None
        self.parent = None  # Adicionado para facilitar a remoção

    def is_full(self):
        return len(self.keys) > MAX_KEYS

    def is_underflow(self):
        """Verifica se o nó tem menos chaves que o mínimo permitido."""
        return len(self.keys) < MIN_KEYS

    def __repr__(self):
        return f"Keys: {self.keys}, Leaf: {self.is_leaf}"

class BPlusTree():
    """Implementação da Estrutura de Índice B+ Tree com Inserção, Busca e Remoção."""
    def __init__(self, order, log_filename='bplustree.log', debugging=False):
        self.order = order
        self.root = BPlusNode(is_leaf=True)
        self._debugging = debugging
        self._config_log(log_filename)
        self._debug('Criando B+ Tree (Ordem = %s)...', order)

    # ----------------------------------------
    # I. INSERÇÃO
    # ----------------------------------------

    def insert(self, record):
        key = record[0]
        self._debug(f"-> Inserindo chave: {key}")
        
        leaf = self._find_leaf(key)
        self._insert_into_leaf(leaf, key, record)

        if leaf.is_full():
            key_up, new_node = self._split(leaf)
            
            if leaf is self.root:
                new_root = BPlusNode(is_leaf=False)
                new_root.keys = [key_up]
                new_root.children = [leaf, new_node]
                leaf.parent = new_root
                new_node.parent = new_root
                self.root = new_root
            else:
                self._insert_into_parent(leaf.parent, key_up, new_node)
        
        self._debug(f"-> Inserção de {key} finalizada.")

    def _insert_into_leaf(self, leaf, key, record):
        temp_items = sorted([(k, r) for k, r in zip(leaf.keys, leaf.children)] + [(key, record)])
        leaf.keys = [item[0] for item in temp_items]
        leaf.children = [item[1] for item in temp_items]

    def _split(self, node):
        mid_point = len(node.keys) // 2
        new_node = BPlusNode(is_leaf=node.is_leaf)
        new_node.parent = node.parent  # Novo irmão divide o mesmo pai

        key_up = node.keys[mid_point]

        if node.is_leaf:
            new_node.keys = node.keys[mid_point:]
            new_node.children = node.children[mid_point:]
            node.keys = node.keys[:mid_point]
            node.children = node.children[:mid_point]
            
            node.next_leaf, new_node.next_leaf = new_node, node.next_leaf
            key_up = new_node.keys[0] # Cópia da chave para cima
        else:
            # Nó Interno: A chave do meio sobe e NÃO fica no nó filho
            new_node.keys = node.keys[mid_point + 1:]
            new_node.children = node.children[mid_point + 1:]
            node.keys = node.keys[:mid_point]
            # O filho que estava à direita da chave que subiu vai para o novo nó
            children_moving = node.children[mid_point + 1:]
            node.children = node.children[:mid_point + 1]
            
            # Atualizar os pais dos filhos que foram movidos
            for child in new_node.children:
                child.parent = new_node

        return key_up, new_node

    def _insert_into_parent(self, parent, key_up, child_node):
        if parent.is_full():
            parent_key_up, new_parent = self._split(parent)
            
            if key_up < parent_key_up:
                target_node = parent
            else:
                target_node = new_parent
            
            self._insert_into_parent_simple(target_node, key_up, child_node)
            
            if parent is self.root:
                new_root = BPlusNode(is_leaf=False)
                new_root.keys = [parent_key_up]
                new_root.children = [parent, new_parent]
                parent.parent = new_root
                new_parent.parent = new_root
                self.root = new_root
            else:
                self._insert_into_parent(parent.parent, parent_key_up, new_parent)
        else:
            self._insert_into_parent_simple(parent, key_up, child_node)

    def _insert_into_parent_simple(self, parent, key, child):
        # Insere ordenado
        idx = 0
        while idx < len(parent.keys) and key > parent.keys[idx]:
            idx += 1
        parent.keys.insert(idx, key)
        parent.children.insert(idx + 1, child)
        child.parent = parent

    # ----------------------------------------
    # II. REMOÇÃO (Implementada com base no seu código de referência)
    # ----------------------------------------

    def remove(self, key):
        """Remove uma chave da árvore."""
        self._debug(f"-> Tentando remover chave: {key}")
        leaf = self._find_leaf(key)
        
        if key not in leaf.keys:
            self._debug("Chave não encontrada.")
            return

        # Remove da folha
        idx = leaf.keys.index(key)
        leaf.keys.pop(idx)
        leaf.children.pop(idx)

        # Verifica Underflow (se a folha ficou muito vazia)
        if leaf == self.root:
            if len(leaf.keys) == 0:
                # Árvore ficou vazia
                self.root = BPlusNode(is_leaf=True)
        elif leaf.is_underflow():
            self._delete_entry(leaf)
        
        self._debug(f"-> Remoção de {key} finalizada.")

    def _delete_entry(self, node):
        """Lida com underflow recursivamente (Merge ou Redistribuição)."""
        
        # Caso base: Se chegamos na raiz
        if node == self.root:
            if len(node.keys) == 0 and len(node.children) > 0:
                # A raiz antiga sumiu, o primeiro filho vira a nova raiz
                self.root = node.children[0]
                self.root.parent = None
            return

        parent = node.parent
        idx = parent.children.index(node)
        
        # Identifica irmãos (Left ou Right)
        sibling = None
        is_left_sibling = False
        
        if idx > 0:
            sibling = parent.children[idx - 1]
            is_left_sibling = True
        elif idx < len(parent.children) - 1:
            sibling = parent.children[idx + 1]
            is_left_sibling = False
        
        if not sibling:
            return # Não deve acontecer em árvore balanceada

        # --- TENTATIVA 1: MERGE (Fusão) ---
        # Verifica se cabe tudo em um só nó
        if len(node.keys) + len(sibling.keys) <= MAX_KEYS:
            self._merge(node, sibling, parent, idx, is_left_sibling)
        
        # --- TENTATIVA 2: REDISTRIBUTION (Empréstimo) ---
        else:
            self._redistribute(node, sibling, parent, idx, is_left_sibling)

    def _merge(self, node, sibling, parent, idx, is_left_sibling):
        self._debug(f"--- Fazendo MERGE ---")
        
        # Sempre fundir o nó da direita (idx maior) para o da esquerda (idx menor)
        if is_left_sibling:
            left_node, right_node = sibling, node
            split_key_idx = idx - 1
        else:
            left_node, right_node = node, sibling
            split_key_idx = idx

        # Pega a chave separadora do pai
        separator_key = parent.keys[split_key_idx]

        if node.is_leaf:
            # Em folhas, apenas copiamos os dados
            left_node.keys.extend(right_node.keys)
            left_node.children.extend(right_node.children)
            left_node.next_leaf = right_node.next_leaf
        else:
            # Em nós internos, descemos a chave separadora do pai
            left_node.keys.append(separator_key)
            left_node.keys.extend(right_node.keys)
            left_node.children.extend(right_node.children)
            # Atualizar pais dos filhos movidos
            for child in right_node.children:
                child.parent = left_node

        # Remove a chave e o ponteiro do filho da direita do pai
        parent.keys.pop(split_key_idx)
        parent.children.pop(split_key_idx + 1) # Remove o right_node

        # Recursão: O pai perdeu uma chave, verifica underflow nele
        if parent.is_underflow():
            self._delete_entry(parent)

    def _redistribute(self, node, sibling, parent, idx, is_left_sibling):
        self._debug(f"--- Fazendo EMPRÉSTIMO (Redistribution) ---")
        
        if node.is_leaf:
            if is_left_sibling:
                # Pega o último do vizinho da esquerda
                borrowed_key = sibling.keys.pop()
                borrowed_val = sibling.children.pop()
                node.keys.insert(0, borrowed_key)
                node.children.insert(0, borrowed_val)
                # Atualiza pai (chave separadora é a nova primeira chave do nó)
                parent.keys[idx - 1] = node.keys[0]
            else:
                # Pega o primeiro do vizinho da direita
                borrowed_key = sibling.keys.pop(0)
                borrowed_val = sibling.children.pop(0)
                node.keys.append(borrowed_key)
                node.children.append(borrowed_val)
                # Atualiza pai (chave separadora é a nova primeira chave do vizinho)
                parent.keys[idx] = sibling.keys[0]
        else:
            # Lógica para nó interno (envolve rotacionar a chave do pai)
            if is_left_sibling:
                # Desce chave do pai, sobe chave do irmão
                separator_idx = idx - 1
                separator_key = parent.keys[separator_idx]
                
                child_to_move = sibling.children.pop()
                key_to_move = sibling.keys.pop()
                
                node.keys.insert(0, separator_key)
                node.children.insert(0, child_to_move)
                child_to_move.parent = node
                
                parent.keys[separator_idx] = key_to_move
            else:
                # Desce chave do pai, sobe chave do irmão
                separator_idx = idx
                separator_key = parent.keys[separator_idx]
                
                child_to_move = sibling.children.pop(0)
                key_to_move = sibling.keys.pop(0)
                
                node.keys.append(separator_key)
                node.children.append(child_to_move)
                child_to_move.parent = node
                
                parent.keys[separator_idx] = key_to_move

    # ----------------------------------------
    # III. BUSCA E UTILITÁRIOS
    # ----------------------------------------

    def search(self, key):
        leaf = self._find_leaf(key)
        found_records = []
        for i, k in enumerate(leaf.keys):
            if k == key:
                found_records.append(leaf.children[i])
            elif k > key:
                break
        return found_records

    def _find_leaf(self, key):
        node = self.root
        while not node.is_leaf:
            child_index = 0
            while child_index < len(node.keys) and key >= node.keys[child_index]:
                child_index += 1
            node = node.children[child_index]
        return node

    def _config_log(self, log):
        self._log = logging.getLogger(log)
        str_format = '%(levelname)s - %(message)s'
        logging.basicConfig(level=logging.INFO, format=str_format) # Simplified basic config
        if self._debugging:
            self._log.setLevel(logging.DEBUG)

    def _debug(self, msg, *args, **kwargs):
        if self._debugging:
            self._log.debug(msg, *args, **kwargs)

    def __repr__(self):
        def print_node(node, level=0):
            indent = "  " * level
            output = f"\n{indent}- Lvl {level} [{'Leaf' if node.is_leaf else 'Node'}]: {node.keys}"
            if not node.is_leaf:
                for child in node.children:
                    output += print_node(child, level + 1)
            return output
        return f"--- B+ Tree (Order: {self.order}) ---" + print_node(self.root)

# --- Teste Final ---
if __name__ == '__main__':
    # Ordem 3: Max 2 chaves. Min 1 chave. Underflow se chaves < 1.
    tree = BPlusTree(order=3, debugging=True)

    data = [(10, "A"), (20, "B"), (5, "C"), (6, "D"), (12, "E"), (30, "F"), (7, "G"), (17, "H")]
    
    print("=== INSERINDO DADOS ===")
    for item in data:
        tree.insert(item)
    print(tree)

    print("\n=== REMOVENDO DADOS ===")
    
    # 1. Remove folha simples (sem underflow)
    print("\n--- Removendo 6 (Simples) ---")
    tree.remove(6)
    print(tree)

    # 2. Remove causando Underflow e Merge
    print("\n--- Removendo 5 (Causa Merge) ---")
    tree.remove(5) # Nó fica vazio -> Merge com vizinho
    print(tree)

    # 3. Remove nó interno (mais complexo)
    print("\n--- Removendo 20 (Causa ajuste interno) ---")
    tree.remove(20)
    print(tree)
    
    # 4. Remove raiz
    print("\n--- Removendo 12 e 30 ---")
    tree.remove(12)
    tree.remove(30)
    print(tree)

DEBUG - Criando B+ Tree Index (Ordem = 3)...
DEBUG - -> Inserindo chave: 10
DEBUG - -> Inserção de 10 finalizada.
DEBUG - -> Inserindo chave: 20
DEBUG - -> Inserção de 20 finalizada.
DEBUG - -> Inserindo chave: 30
DEBUG - --- Fazendo SPLIT no nó: [10, 20, 30] ---
DEBUG - --- Nova raiz criada com chave: 20 ---
DEBUG - -> Inserção de 30 finalizada.
DEBUG - -> Inserindo chave: 5
DEBUG - -> Inserção de 5 finalizada.
DEBUG - -> Inserindo chave: 25



--- INSERÇÕES ---


DEBUG - --- Fazendo SPLIT no nó: [20, 25, 30] ---
DEBUG - -> Inserção de 25 finalizada.
DEBUG - -> Inserindo chave: 15
DEBUG - --- Fazendo SPLIT no nó: [5, 10, 15] ---
DEBUG - -> Inserção de 15 finalizada.
DEBUG - -> Inserindo chave: 35
DEBUG - --- Fazendo SPLIT no nó: [25, 30, 35] ---
DEBUG - --- Fazendo SPLIT no nó: [10, 20, 25] ---
DEBUG - -> Inserção de 35 finalizada.
DEBUG - -> Buscando chave: 20
DEBUG - Chave 20 não encontrada.
DEBUG - -> Buscando chave: 5
DEBUG - Encontrado 1 registro(s) para a chave 5.
DEBUG - -> Buscando chave: 40
DEBUG - Chave 40 não encontrada.



--- Árvore B+ (Ordem: 3) ---
- Nível 0 (Leaf: False): Chaves: [20]
  - Nível 1 (Leaf: False): Chaves: [10]
    - Nível 2 (Leaf: True): Chaves: [5]
  - Nível 1 (Leaf: False): Chaves: [25, 30]
    - Nível 2 (Leaf: True): Chaves: [10, 15]
    - Nível 2 (Leaf: True): Chaves: [20]
    - Nível 2 (Leaf: True): Chaves: [30, 35]
    - Nível 2 (Leaf: True): Chaves: [25]

--- BUSCAS ---
Resultado da busca por 20: []
Resultado da busca por 5: [(5, 'D')]
Resultado da busca por 40 (Não existe): []
